Overall functions count:
- filter: 3 -> 5
- join: 2 -> 6
- group by: 2 -> 5
- order by: _ -> 6
- window function: 2 -> 3

In [1]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("IMDB Queries").getOrCreate()
spark = (SparkSession.builder.appName("IMDB Queries")
         .config("spark.executor.memory", "4g")
         .config("spark.driver.memory", "4g")
         .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 13:06:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os

In [173]:
data_dir = "../data"
title_basics_path = os.path.abspath(f"{data_dir}/title.basics.cleaned.tsv")
title_ratings_path = os.path.abspath(f"{data_dir}/title.ratings.cleaned.tsv")
title_akas_path = os.path.abspath(f"{data_dir}/title.akas.cleaned.tsv")
title_crew_path = os.path.abspath(f"{data_dir}/title.crew.cleaned.tsv")
title_crew_path = os.path.abspath(f"{data_dir}/title.crew.cleaned.tsv")
name_basics_path = os.path.abspath(f"{data_dir}/name.basics.cleaned.tsv")
title_principals_path = os.path.abspath(f"{data_dir}/title.principals.cleaned.tsv")

title_crew = spark.read.csv(title_crew_path, header=True, sep="\t")
title_basics = spark.read.csv(title_basics_path, header=True, sep="\t")
title_ratings = spark.read.csv(title_ratings_path, header=True, sep="\t")
title_akas = spark.read.csv(title_akas_path, header=True, sep="\t")
name_basics = spark.read.csv(name_basics_path, header=True, sep="\t")
title_principals = spark.read.csv(title_principals_path, header=True, sep="\t")

### Query 1: Визначити топ-N країн, фільми яких мають найвищу середню оцінку? (+ відфільтрувати за мінімальною кількістю голосів)

Визначити топ-N країн, фільми яких мають найвищу середню оцінку?
   (можна ще відфільтрувати за мінімальною кількістю голосів)
   - Таблиці: title.basics, title.ratings, title.akas
   - Операції: join, group by, filter, order by

In [5]:
import os
from pyspark.sql.functions import avg, desc
from pyspark.sql.functions import split, explode, col, avg, desc

In [8]:
# data_dir = "../data"
# title_basics_path = os.path.abspath(f"{data_dir}/title.basics.cleaned.tsv")
# title_ratings_path = os.path.abspath(f"{data_dir}/title.ratings.cleaned.tsv")
# title_akas_path = os.path.abspath(f"{data_dir}/title.akas.cleaned.tsv")

# title_basics = spark.read.csv(title_basics_path, header=True, sep="\t")
# title_ratings = spark.read.csv(title_ratings_path, header=True, sep="\t")
# title_akas = spark.read.csv(title_akas_path, header=True, sep="\t")

In [6]:
movies = title_basics.filter(col("titleType") == "movie")
movies.show(3)

+---------+---------+--------------------+--------------------+-------+----------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult| startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+----------+-------+--------------+--------------------+
|tt0000009|    movie|          Miss Jerry|          Miss Jerry|      0|1894-01-01|   null|            45|             Romance|
|tt0000147|    movie|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|1897-01-01|   null|           100|Documentary,News,...|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|1906-01-01|   null|            70|Action,Adventure,...|
+---------+---------+--------------------+--------------------+-------+----------+-------+--------------+--------------------+
only showing top 3 rows



In [7]:
movies_with_region = movies.join(title_akas.select("titleId", "region"), 
                                 movies["tconst"] == title_akas["titleId"], 
                                 how="inner").drop(title_akas["titleId"])

movies_with_ratings = movies_with_region.join(title_ratings, on="tconst", how="inner") \
    .withColumn("averageRating", col("averageRating").cast("double")) \
    .withColumn("numVotes", col("numVotes").cast("int"))

In [9]:
min_votes = 1000
filtered = movies_with_ratings.filter(col("numVotes") >= min_votes)
filtered.show(3)

[Stage 13:>                                                         (0 + 1) / 1]

+---------+---------+------------+--------------+-------+----------+-------+--------------+--------------------+-------+-------------+--------+
|   tconst|titleType|primaryTitle| originalTitle|isAdult| startYear|endYear|runtimeMinutes|              genres| region|averageRating|numVotes|
+---------+---------+------------+--------------+-------+----------+-------+--------------+--------------------+-------+-------------+--------+
|tt0002423|    movie|     Passion|Madame DuBarry|      0|1919-01-01|   null|           113|Biography,Drama,R...|Unknown|          6.6|    1063|
|tt0002423|    movie|     Passion|Madame DuBarry|      0|1919-01-01|   null|           113|Biography,Drama,R...|     US|          6.6|    1063|
|tt0002423|    movie|     Passion|Madame DuBarry|      0|1919-01-01|   null|           113|Biography,Drama,R...|     SE|          6.6|    1063|
+---------+---------+------------+--------------+-------+----------+-------+--------------+--------------------+-------+-------------+--

In [10]:
avg_ratings_by_country = filtered.groupBy("region").agg(
    avg("averageRating").alias("avg_rating")
).orderBy(desc("avg_rating"))

N = 10
top_n_countries = avg_ratings_by_country.limit(N)

In [11]:
top_n_countries.show(N, truncate=False)

[Stage 21:====================================================>   (16 + 1) / 17]

+------+------------------+
|region|avg_rating        |
+------+------------------+
|LY    |8.399999999999999 |
|JO    |7.8               |
|UG    |7.55              |
|MM    |7.5               |
|NP    |7.451515151515151 |
|SD    |7.449999999999999 |
|CSXX  |7.35              |
|PS    |7.290909090909091 |
|DDDE  |7.1808219178082195|
|BT    |7.1571428571428575|
+------+------------------+



In [12]:
region_mapping = {
    "LY": "Libya",
    "JO": "Jordan",
    "UG": "Uganda",
    "MM": "Myanmar",
    "NP": "Nepal",
    "SD": "Sudan",
    "CSXX": "Czech Republic",
    "PS": "Palestine",
    "DDDE": "Germany",
    "BT": "Bhutan"
}

broadcast_region_mapping = spark.sparkContext.broadcast(region_mapping)

In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def get_country_name(region_code):
    return broadcast_region_mapping.value.get(region_code, "Unknown")

get_country_name_udf = udf(get_country_name, StringType())
expanded_regions = top_n_countries.withColumn("country", get_country_name_udf(col("region")))

In [14]:
expanded_regions.show(N, truncate=False)

[Stage 33:====================================================>   (16 + 1) / 17]

+------+------------------+--------------+
|region|avg_rating        |country       |
+------+------------------+--------------+
|LY    |8.399999999999999 |Libya         |
|JO    |7.8               |Jordan        |
|UG    |7.55              |Uganda        |
|MM    |7.5               |Myanmar       |
|NP    |7.451515151515151 |Nepal         |
|SD    |7.449999999999999 |Sudan         |
|CSXX  |7.35              |Czech Republic|
|PS    |7.290909090909091 |Palestine     |
|DDDE  |7.1808219178082195|Germany       |
|BT    |7.1571428571428575|Bhutan        |
+------+------------------+--------------+



### Query 2: Які фільми отримали найкращу оцінку від користувачів у кожному десятилітті?

Які фільми отримали найбільшу кількість голосів користувачів (за рейтингом або кількістю відгуків) у кожному десятилітті?
   - Таблиці: title.basics, title.ratings
   - Операції: join, order by, filter, window

In [184]:
from pyspark.sql.functions import col, floor, desc, row_number
from pyspark.sql.window import Window

In [189]:
movies = title_basics.filter(col("titleType") == "movie") \
    .withColumn("startYear", substring("startYear", 1, 4).cast("int")) \
    .filter(col("startYear").isNotNull()) \
    .join(title_ratings, on="tconst", how="inner")

movies_with_decade = movies.withColumn("decade", (floor(col("startYear") / 10) * 10).cast("int"))

In [192]:
window_spec = Window.partitionBy("decade").orderBy(desc("averageRating"), desc("numVotes"))

top_movies_per_decade = movies_with_decade.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("primaryTitle", "decade", "startYear", "averageRating", "numVotes")

top_movies_per_decade.orderBy("decade").show(truncate=False)

[Stage 1526:========>                                               (1 + 6) / 7]

+---------------------------+------+---------+-------------+--------+
|primaryTitle               |decade|startYear|averageRating|numVotes|
+---------------------------+------+---------+-------------+--------+
|Miss Jerry                 |1890  |1894     |5.4          |223     |
|België                     |1900  |1909     |7.4          |28      |
|Powder                     |1910  |1916     |9.0          |16      |
|Chhatrapati Sambhaji       |1920  |1924     |9.3          |18      |
|Dhrubo                     |1930  |1934     |9.6          |6       |
|The College                |1940  |1943     |9.9          |65      |
|Greenland                  |1950  |1952     |9.4          |5       |
|Pinagbuklod ng langit      |1960  |1969     |9.5          |10      |
|Against Reason and by Force|1970  |1974     |9.6          |5       |
|Csonka Bereg               |1980  |1989     |9.8          |6       |
|Juventud en drogas         |1990  |1996     |9.8          |8       |
|Kot ptic           

In [193]:
window_spec = Window.partitionBy("decade").orderBy(desc("numVotes"), desc("averageRating"))

top_movies_per_decade_2 = movies_with_decade.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("primaryTitle", "decade", "startYear", "averageRating", "numVotes")

top_movies_per_decade_2.orderBy("decade").show(truncate=False)

[Stage 1535:================================================>       (6 + 1) / 7]

+-------------------------------+------+---------+-------------+--------+
|primaryTitle                   |decade|startYear|averageRating|numVotes|
+-------------------------------+------+---------+-------------+--------+
|Jeffries-Sharkey Contest       |1890  |1899     |3.8          |81      |
|The Story of the Kelly Gang    |1900  |1906     |6.0          |980     |
|Back to God's Country          |1910  |1919     |6.2          |993     |
|Lights of New York             |1920  |1928     |5.7          |999     |
|Mayerling                      |1930  |1936     |7.0          |996     |
|Kiss of Death                  |1940  |1947     |7.4          |9998    |
|The Conquest of Everest        |1950  |1953     |7.3          |998     |
|Vodka, Mr. Palmu               |1960  |1969     |5.9          |999     |
|The Hanging Woman              |1970  |1973     |5.7          |998     |
|Bad Timing: A Sensual Obsession|1980  |1980     |6.9          |9994    |
|Foxfire                        |1990 

### Query 3: Які режисери мають найвищу середню оцінку серед фільмів, випущених у різних жанрах? (+ відфільтрувати за мінімальною кількістю знятих фільмів (n=15))

Які режисери мають найвищу середню оцінку серед фільмів, випущених у різних жанрах?
   (можливо ще відфільтрувати за мінімальною кількістю знятих фільмів (n=15))
   - Таблиці: title.crew, title.ratings, title.basics, name.basics
   - Операції: join, group by, filter, window, order by

In [34]:
from pyspark.sql.functions import col, explode, split, avg, count, row_number
from pyspark.sql.window import Window

In [18]:
movies = title_basics.filter((col("titleType") == "movie"))

In [20]:
movies_with_ratings = movies.join(title_ratings, on="tconst")

movies_with_directors = movies_with_ratings.join(title_crew.select("tconst", "directors"), on="tconst")

In [35]:
movies_exploded = movies_with_directors \
    .withColumn("director", explode(split(col("directors"), ","))) \
    .withColumn("genre", explode(split(col("genres"), ","))) \
    .filter(col("genre").isNotNull())

director_genre_stats = movies_exploded.groupBy("director", "genre") \
    .agg(
        avg(col("averageRating")).alias("avg_rating"),
        count("*").alias("movie_count")
    ) \
    .filter(col("movie_count") >= 15)

director_genre_named = director_genre_stats.join(
    name_basics.select("nconst", "primaryName"),
    director_genre_stats.director == name_basics.nconst,
    "left"
)

genre_window = Window.partitionBy("genre").orderBy(col("avg_rating").desc())
ranked = director_genre_named.withColumn("rank", row_number().over(genre_window))

top_directors_unique_genres = ranked.filter(col("rank") == 1) \
    .select("primaryName", "genre", "avg_rating", "movie_count") \
    .orderBy(col("avg_rating").desc())

In [37]:
top_directors_unique_genres.show(25, truncate=False)

[Stage 157:===================================================>   (16 + 1) / 17]

+-------------------------+-----------+------------------+-----------+
|primaryName              |genre      |avg_rating        |movie_count|
+-------------------------+-----------+------------------+-----------+
|Sibtain Fazli            |Drama      |9.066666666666666 |15         |
|Sergey A.                |Documentary|8.945454545454545 |22         |
|Alonso O. Lara           |Crime      |8.725             |16         |
|Mariano Alvarez          |Action     |8.388235294117647 |17         |
|Claudio Costa            |Biography  |8.305263157894737 |19         |
|Claudio Costa            |History    |8.105             |20         |
|P. Ramlee                |Comedy     |7.983333333333334 |18         |
|Paul Dugdale             |Music      |7.927272727272728 |22         |
|K. Viswanath             |Musical    |7.773684210526315 |19         |
|William Higgins          |Adult      |7.75              |24         |
|Chuck Jones              |Animation  |7.48              |15         |
|Willi

### Query 4: Як змінювалося середня оцінка для усіх фільмів для режисерів (топ-N за кількістю фільмів або за найвищим середнім рейтингом) протягом кожних 5-ти років їхньої кар'єри (порівняно з їхньою загальною середньою оцінкою за всю кар'єру)?

Як змінювалося середня оцінка для усіх фільмів для режисерів (топ-N за кількістю фільмів або за найвищим середнім рейтингом) протягом кожних 5-ти років їхньої кар'єри (порівняно з їхньою загальною середньою оцінкою за всю кар'єру)? +
   * Взяти середній рейтинг першого фільму (рейтинг на початку кар'єри) та загальний рейтинг (середній рейтинг режисера за всю кар'єру). Також рахувати середній рейтинг фільмів, знятих кожні 5 років протягом усієї кар'єри режисера
   - Таблиці: title.crew, title.basics, title.ratings
   - Операції: join, group by, window, filters, order by

In [154]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, substring, avg, count, floor, split, explode, min, max, lit, first, last, format_number, when, lag)
from pyspark.sql.window import Window

In [2]:
spark = (SparkSession.builder.appName("Directors productivity analysis")
         .config("spark.executor.memory", "4g")
         .config("spark.driver.memory", "4g")
         .getOrCreate())

# data_dir = "../data"
# title_crew_path = os.path.abspath(f"{data_dir}/title.crew.cleaned.tsv")
# title_basics_path = os.path.abspath(f"{data_dir}/title.basics.cleaned.tsv")
# title_ratings_path = os.path.abspath(f"{data_dir}/title.ratings.cleaned.tsv")

# title_crew = spark.read.csv(title_crew_path, header=True, sep="\t")
# title_basics = spark.read.csv(title_basics_path, header=True, sep="\t")
# title_ratings = spark.read.csv(title_ratings_path, header=True, sep="\t")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/06 21:06:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [148]:
movies = (title_basics
          .filter(col("titleType") == "movie")
          .withColumn("startYear", substring("startYear", 1, 4).cast("int"))
          .select("tconst", "startYear"))

movies_with_directors = title_crew.select("tconst", "directors")
ratings = title_ratings.select("tconst", "averageRating")

director_movies = (movies_with_directors
                   .withColumn("director_nconst", explode(split(col("directors"), ",")))
                   .join(name_basics, name_basics.nconst == col("director_nconst"))
                   .select("tconst", "director_nconst", "primaryName"))

director_movies = director_movies.join(movies, "tconst")
director_movies = director_movies.join(ratings, "tconst")

top_n_directors = (director_movies
                   .groupBy("primaryName")
                   .agg(count("tconst").alias("film_count"))
                   .orderBy(col("film_count").desc())
                   .limit(10))

director_movies = director_movies.join(top_n_directors, director_movies.primaryName == top_n_directors.primaryName).drop(top_n_directors.primaryName)

In [149]:
director_movies.show(3)

+---------+---------------+---------+-------------+--------------+----------+
|   tconst|director_nconst|startYear|averageRating|   primaryName|film_count|
+---------+---------------+---------+-------------+--------------+----------+
|tt0007910|      nm0002031|     1918|          6.5|Michael Curtiz|       156|
|tt0009322|      nm0002031|     1919|          4.0|Michael Curtiz|       156|
|tt0009622|      nm0002031|     1918|          4.3|Michael Curtiz|       156|
+---------+---------------+---------+-------------+--------------+----------+
only showing top 3 rows



In [150]:
first_last_years = (director_movies
                    .groupBy("primaryName")
                    .agg(min("startYear").alias("firstYear"), max("startYear").alias("lastYear")))

director_movies = director_movies.join(first_last_years, "primaryName")
director_movies.show(10)

[Stage 661:===================================>                   (11 + 6) / 17]

+--------------+---------+---------------+---------+-------------+----------+---------+--------+
|   primaryName|   tconst|director_nconst|startYear|averageRating|film_count|firstYear|lastYear|
+--------------+---------+---------------+---------+-------------+----------+---------+--------+
|Michael Curtiz|tt0007910|      nm0002031|     1918|          6.5|       156|     1913|    1961|
|Michael Curtiz|tt0009322|      nm0002031|     1919|          4.0|       156|     1913|    1961|
|Michael Curtiz|tt0009622|      nm0002031|     1918|          4.3|       156|     1913|    1961|
|Michael Curtiz|tt0011011|      nm0002031|     1920|          4.6|       156|     1913|    1961|
|Michael Curtiz|tt0014171|      nm0002031|     1923|          4.6|       156|     1913|    1961|
|Michael Curtiz|tt0019917|      nm0002031|     1929|          4.9|       156|     1913|    1961|
|Michael Curtiz|tt0026418|      nm0002031|     1935|          6.0|       156|     1913|    1961|
|Michael Curtiz|tt0026632|    

In [151]:
# career_window = Window.partitionBy("director")
# director_movies = (director_movies
#                    .withColumn("averageRating", avg("averageRating").over(career_window).cast("double")))
#
# ratings_summary = (director_movies
#                    .select("director",
#                            format_number(col("firstFilmRating"), 2).alias("firstFilmRating"),
#                            format_number(col("lastFilmRating"), 2).alias("lastFilmRating"))
#                            # format_number(col("averageRating"), 2).alias("averageRating"))
#                    .dropDuplicates(["director"]))
# ratings_summary.show(10)

In [156]:
def round_to_5_years(year_col):
    # return col(year_col)
    return floor(col(year_col) / 5) * 5

director_movies = director_movies.withColumn("careerPeriod", round_to_5_years("startYear"))
periodic_avg_ratings = (director_movies
                        .groupBy("primaryName", "careerPeriod")
                        .agg(avg("averageRating").alias("periodicAvgRating")))

min_window = director_movies.agg(min("careerPeriod")).collect()[0][0]
max_window = director_movies.agg(max("careerPeriod")).collect()[0][0]

five_years = list(range(min_window, max_window + 5, 5))
five_years_df = spark.createDataFrame([(d,) for d in five_years], ["careerPeriod"])

full_director_five_years = (top_n_directors
                            .crossJoin(five_years_df)
                            .join(periodic_avg_ratings, ["primaryName", "careerPeriod"], "left"))

director_movies_pivoted = (full_director_five_years
                           .groupBy("primaryName", "film_count")
                           .pivot("careerPeriod", five_years)
                           .agg(avg("periodicAvgRating"))
                           .na.fill("-"))

for year in range(min_window, max_window + 5, 5):
    col_name = f"{year}"
    formatted_col = f"{year}_{(year - (year // 100) * 100) + 4}"
    director_movies_pivoted = (
        director_movies_pivoted
        .withColumnRenamed(col_name, formatted_col)
        .withColumn(formatted_col, format_number(col(formatted_col), 2))
        .withColumn(formatted_col, when(col(formatted_col).isNotNull(), col(formatted_col)).otherwise(lit("-"))))

director_movies_pivoted = director_movies_pivoted.na.fill("-")

year_columns = director_movies_pivoted.columns[2:]
long_df = director_movies_pivoted.select(
    'primaryName',
    F.explode(F.array(
        *[F.struct(F.lit(year).alias("year"), F.col(year).alias("rating")) for year in year_columns])).alias(
        "year_rating")
).select(
    'year_rating.year',
    'primaryName',
    'year_rating.rating'
)

pivoted_df = long_df.groupBy('year').pivot('primaryName').agg(F.first('rating'))

In [158]:
pivoted_df.show(3)

+-------+-------------------------+----------+------------+---------------+-----------+--------------+--------------+--------------+------------+----------------+
|   year|Gilberto Martínez Solares|Godfrey Ho|Jesús Franco|Lesley Selander|Lew Landers|Michael Curtiz|Osman F. Seden|Richard Thorpe|Sam Newfield|William Beaudine|
+-------+-------------------------+----------+------------+---------------+-----------+--------------+--------------+--------------+------------+----------------+
|1910_14|                        -|         -|           -|              -|          -|          4.07|             -|             -|           -|               -|
|1915_19|                        -|         -|           -|              -|          -|          4.32|             -|             -|           -|               -|
|1920_24|                        -|         -|           -|              -|          -|          4.98|             -|          4.38|           -|            7.00|
+-------+-------------

In [160]:
print(director_movies_pivoted.columns)

['primaryName', 'film_count', '1910_14', '1915_19', '1920_24', '1925_29', '1930_34', '1935_39', '1940_44', '1945_49', '1950_54', '1955_59', '1960_64', '1965_69', '1970_74', '1975_79', '1980_84', '1985_89', '1990_94', '1995_99', '2000_4', '2005_9', '2010_14']


In [161]:
print(pivoted_df.columns)

['year', 'Gilberto Martínez Solares', 'Godfrey Ho', 'Jesús Franco', 'Lesley Selander', 'Lew Landers', 'Michael Curtiz', 'Osman F. Seden', 'Richard Thorpe', 'Sam Newfield', 'William Beaudine']


In [164]:
pivoted_df_cleaned = pivoted_df
for col_name in pivoted_df.columns:
    cleaned_name = col_name.replace(" ", "_").replace(".", "")
    pivoted_df_cleaned = pivoted_df_cleaned.withColumnRenamed(col_name, cleaned_name)

result_df = pivoted_df_cleaned.select(
    'year',
    *[
        F.when(
            (F.col(director) == '-') & (F.lag(F.col(director)).over(window_spec) == '-'),
            '-'
        ).when(
            (F.col(director) != '-') & (F.lag(F.col(director)).over(window_spec) == '-'),
            F.col(director)
        ).when(
            (F.col(director) != '-') & (F.lag(F.col(director)).over(window_spec) != '-'),
            F.round(F.col(director) - F.lag(F.col(director)).over(window_spec), 2)
        ).otherwise(F.col(director)).alias(director)
        for director in pivoted_df_cleaned.columns[1:]
    ]
)

In [170]:
pivoted_df_cleaned = result_df.withColumnRenamed("Gilberto_Martínez_Solares", "Gilberto_Mart...")
pivoted_df_cleaned.show(50, truncate=False)

+-------+----------------+----------+------------+---------------+-----------+--------------+-------------+--------------+------------+----------------+
|year   |Gilberto_Mart...|Godfrey_Ho|Jesús_Franco|Lesley_Selander|Lew_Landers|Michael_Curtiz|Osman_F_Seden|Richard_Thorpe|Sam_Newfield|William_Beaudine|
+-------+----------------+----------+------------+---------------+-----------+--------------+-------------+--------------+------------+----------------+
|1910_14|-               |-         |-           |-              |-          |4.07          |-            |-             |-           |-               |
|1915_19|-               |-         |-           |-              |-          |0.25          |-            |-             |-           |-               |
|1920_24|-               |-         |-           |-              |-          |0.66          |-            |4.38          |-           |7.00            |
|1925_29|-               |-         |-           |-              |-          |0.2 

### Query 5: Які жанри мають найвищий середній рейтинг (середня оцінка) та як змінювався їхній середній рейтинг протягом кожного десятиліття?

Які жанри мають найвищий середній рейтинг (середня оцінка) та як змінювався їхній середній рейтинг протягом кожного десятиліття? +-
   - Таблиці: title.ratings, title.basics
   - Операції: join, group by, order by

In [126]:
from pyspark.sql.functions import col, explode, split, avg, count, floor, substring, when, lit

In [123]:
title_basics_trimmed = title_basics.withColumn("tconst", trim(col("tconst")))
title_ratings_trimmed = title_ratings.withColumn("tconst", trim(col("tconst")))

movies = title_basics_trimmed.filter(col("titleType") == "movie")\
          .withColumn("startYear", substring("startYear", 1, 4).cast("int"))
joined_df = movies.join(title_ratings_trimmed, on="tconst", how="inner")

In [124]:
joined_df = joined_df \
    .withColumn("genre", explode(split(col("genres"), ","))) \
    .withColumn("startYear", when(col("startYear").isNotNull(), col("startYear").cast("int"))) \
    .withColumn("decade", floor(col("startYear") / 10) * 10)

In [129]:
genre_by_decade = joined_df.groupBy("genre", "decade") \
    .agg(avg("averageRating").alias("avg_rating"))

decade_values = genre_by_decade.select("decade").distinct().orderBy("decade") \
    .rdd.flatMap(lambda x: x).collect()

genre_pivoted = genre_by_decade.groupBy("genre") \
    .pivot("decade", decade_values) \
    .agg(avg("avg_rating")) \
    .orderBy("genre")

formatted_df = genre_pivoted
for decade in decade_values:
    decade_str = str(decade)
    if decade_str in formatted_df.columns:
        formatted_df = formatted_df.withColumn(
            decade_str,
            when(col(decade_str).isNull(), lit("-"))
            .otherwise(format_number(col(decade_str), 2))
            .cast("string")
        )
columns = formatted_df.columns
formatted_df = formatted_df.drop(columns[1])

formatted_df.show(truncate=False)

[Stage 547:========>                                                (1 + 6) / 7]

+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|genre      |1890|1900|1910|1920|1930|1940|1950|1960|1970|1980|1990|2000|2010|2020|
+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|Action     |-   |4.90|5.85|5.80|5.84|6.03|6.11|6.01|5.87|5.58|5.54|5.56|5.73|5.90|
|Adult      |-   |-   |-   |-   |-   |-   |-   |5.56|5.19|5.69|6.18|5.79|6.14|6.52|
|Adventure  |-   |4.60|5.48|5.48|5.80|5.92|5.87|5.80|5.77|5.68|5.80|5.76|6.04|6.02|
|Animation  |-   |-   |6.00|6.03|6.34|6.35|6.56|6.51|6.48|6.60|6.46|6.24|6.23|6.41|
|Biography  |-   |6.15|5.94|6.29|6.37|6.46|6.51|6.63|6.63|6.68|6.88|6.95|7.09|6.90|
|Comedy     |-   |3.97|6.06|6.14|6.15|6.16|6.03|5.92|5.64|5.74|5.86|5.85|5.86|6.02|
|Crime      |-   |-   |6.09|5.90|6.06|6.20|6.16|6.04|5.86|5.79|5.84|5.91|5.98|6.13|
|Documentary|3.90|4.72|6.26|6.49|6.38|6.39|6.58|6.73|6.93|7.19|7.17|7.27|7.22|7.27|
|Drama      |-   |4.94|6.01|6.15|6.14|6.27|6.32|6.23|6.13|6.07|6.16|6.20|6.2

### Query 6: Які актори найчастіше з'являються у найнерейтинговіших фільмах (із середньою оцінкою нижче 5.0), але які при цьому входять до топ-N фільмів за кількістю відгуків?

Які актори найчастіше з’являються у найнерейтинговіших фільмах (за середньою оцінкою)? 
Які актори найчастіше з'являються у найнерейтинговіших фільмах (із середньою оцінкою нижче 5.0), але які при цьому входять до топ-N фільмів за кількістю відгуків?
   - Таблиці: title.ratings, title.principals, title.basics, name.basics
   - Операції: join, group by, filter, order by

In [174]:
movies = title_basics.filter(col("titleType") == "movie")

low_rated_movies = movies.join(title_ratings, on="tconst") \
                         .filter(col("averageRating") < 5.0)

In [181]:
N = 500
top_n_low_rated = low_rated_movies.orderBy(col("numVotes").desc()).limit(N)

In [182]:
actors = title_principals.filter(col("category").isin("actor", "actress"))

actors_in_top_n_bad_movies = top_n_low_rated.join(actors, on="tconst")
actors_with_names = actors_in_top_n_bad_movies.join(name_basics, on="nconst")

actor_counts = actors_with_names.groupBy("nconst", "primaryName") \
                                .count() \
                                .orderBy(desc("count"))

In [183]:
pivoted_df_cleaned = result_df.withColumnRenamed("count", "Films with low rating but top number of votes")
actor_counts.show(20, truncate=False)

+---------+------------------+-----+
|nconst   |primaryName       |count|
+---------+------------------+-----+
|nm0364490|Brad Harris       |4    |
|nm0353624|Dan Haggerty      |4    |
|nm0434580|Michael Kagan     |3    |
|nm0184392|Martin Kove       |3    |
|nm0637508|Zachi Noy         |3    |
|nm0596701|Paul Naschy       |3    |
|nm0441955|Yftach Katzur     |3    |
|nm0039068|John Ashley       |3    |
|nm0001142|Uschi Digard      |3    |
|nm0006763|Jackie Shroff     |3    |
|nm0505323|Johny Lever       |3    |
|nm0868692|José Torres       |3    |
|nm0930611|Freeman Williams  |3    |
|nm1102326|Gino Turini       |3    |
|nm0595110|Colin Mochrie     |3    |
|nm0823155|Don Stark         |2    |
|nm0370827|Ron Haydock       |2    |
|nm0205791|Ivano Davoli      |2    |
|nm0740067|Maurice Ronet     |2    |
|nm0021251|Alexander Allerson|2    |
+---------+------------------+-----+
only showing top 20 rows

